In [1]:
import os
import pandas as pd

In [2]:
os.getcwd()

'd:\\Studies\\Materials\\Second-cycle\\I year\\II trimester\\ML and StatL\\ML\\ML-project\\src'

In [3]:
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')
mushrooms.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w


In [4]:
mushrooms.dtypes

class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-root                object
stem-surface             object
stem-color               object
veil-type                object
veil-color               object
has-ring                 object
ring-type                object
spore-print-color        object
habitat                  object
season                   object
dtype: object

In [5]:
n_rows = mushrooms.shape[0]
n_columns = mushrooms.shape[1]

print(f'There are {n_rows} rows and {n_columns} columns.')

There are 61069 rows and 21 columns.


In [6]:
print("There are no missing values." if not mushrooms.isnull().any().any()\
      else "There are missing values in the dataframe.")

There are missing values in the dataframe.


In [7]:
nan_counts = mushrooms.isna().sum()
nan_counts[nan_counts > 0]

cap-surface          14120
gill-attachment       9884
gill-spacing         25063
stem-root            51538
stem-surface         38124
veil-type            57892
veil-color           53656
ring-type             2471
spore-print-color    54715
dtype: int64

In [8]:
edible_count = (mushrooms['class'] == 'e').sum()
poisonous_count = mushrooms.shape[0] - edible_count
print(f"Number of edible mushrooms: {edible_count}\nNumber of poisonous mushrooms: {poisonous_count}")

Number of edible mushrooms: 27181
Number of poisonous mushrooms: 33888


In [30]:
mushrooms_1 = mushrooms.dropna(axis=1)

X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

In [31]:
X

array([[15.26, 'x', 'o', ..., 't', 'd', 'w'],
       [16.6, 'x', 'o', ..., 't', 'd', 'u'],
       [14.07, 'x', 'o', ..., 't', 'd', 'w'],
       ...,
       [1.27, 's', 'y', ..., 'f', 'd', 'u'],
       [1.24, 'f', 'y', ..., 'f', 'd', 'u'],
       [1.17, 's', 'y', ..., 'f', 'd', 'u']], dtype=object)

In [32]:
y

array(['p', 'p', 'p', ..., 'p', 'p', 'p'], dtype=object)

In [36]:
from src.UDFs import DecisionTreeClassifier
import numpy as np

tree = DecisionTreeClassifier()
tree.fit(X, y)

ModuleNotFoundError: No module named 'src'